In [1]:
import pandas as pd

In [2]:
import re

In [3]:
import csv

In [4]:
train_df = pd.read_csv("train.csv")

In [5]:
classes_columns = [c for c in train_df.columns if c not in ["comment_text", "id"]]

In [6]:
train_df["toxic"] = train_df[classes_columns].any(axis=1).astype(int)

In [7]:
# exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable . 
# linebreaks, contractions

ip_regex = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")
time_regex = re.compile("")
symbols_regex = re.compile("([.?!,:\"]+)")
spaces_regex = re.compile("\s+")
brackets_regex = re.compile("[()]")

def preprocess_comment(comment):
    comment = ip_regex.sub("", comment)
    comment = comment.replace("\n", " ").lower()
    comment = symbols_regex.sub(" \\1 ", comment)
    comment = brackets_regex.sub(" ", comment)
    comment = spaces_regex.sub(" ", comment)
    return comment.strip()

In [8]:
train_df["preprocessed_comment"] = train_df["comment_text"].apply(preprocess_comment)

In [9]:
train_df["length"] = train_df["preprocessed_comment"].apply(lambda x: len(x))

In [10]:
train_df = train_df[train_df["length"] <= 400].copy()

# Translate posts

In [11]:
import requests
from tqdm import tqdm_notebook
import os

In [72]:
YANDEX_SESSION_API_KEY = "6e35a458.5c7bbf99.b921cfd7-1-0" #os.environ["YANDEX_SESSION_API_KEY"]
YANDEX_URL ="https://translate.yandex.net/api/v1/tr.json/translate?id={}&srv=tr-text&lang=en-ru&reason=auto"

In [108]:
def translate(text):
    url = YANDEX_URL.format(YANDEX_SESSION_API_KEY)
    response = requests.post(url, data={"text": text, "options": 4}).json()
    if "message" in response:
        print(response["message"])
    return response["text"]

In [109]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [80]:
index = 0
end_of_comment = "EOCOMMENT\n"
all_translations = []
all_texts = train_df["comment_text"].tolist()

In [88]:
index

48959

In [112]:
index

88678

In [ ]:
for text in tqdm_notebook(all_texts[index:]):
    translation = translate(text[0:600])
    index += 1
#     if (index % 1000) == 0:
    all_translations.append(translation)

In [113]:
with open('data/rus_toxicity_1.all', "w") as file:
    for comment in all_translations:
        file.write(comment[0] + '\n')

In [15]:
translate()

['Объяснение\nПочему изменения, сделанные под моим логином хардкорный фанат Металлики были отменены? Они не вандализм, просто катарсис какой-то газ после того, как я проголосовал в Нью-Йорке куклы ВСК. И, пожалуйста, не удаляйте шаблон на странице разговора, так как я сейчас на пенсии.89.205.38.27']

In [16]:
texts = [get_translate(text) for text in tqdm_notebook(train_df["comment_text"])]

KeyboardInterrupt: 

# Save dataset

In [13]:
with open('data/toxicity.pos', "w") as file:
    for comment in train_df[train_df["toxic"] == 1]["preprocessed_comment"]:
        file.write(comment + '\n')

In [14]:
with open('data/toxicity.neg', "w") as file:
    for comment in train_df[train_df["toxic"] == 0]["preprocessed_comment"]:
        file.write(comment + '\n')